# Educational Toolkit for Mapping Large Ensembles

This is an educational toolkit to demonstrate how to deal with CESM1 data. 

Things to note: 

- We are using the Ocean model from CESM1
- The data is hosted on Amazon Web Services (AWS), however this data is freely available so no account for AWS is necessary.
- We are investigating different experiments through the Ocean Model.
- We are only looking at few variables. All variables available through the CESM1 data are found on the link below: 
https://www.cesm.ucar.edu/community-projects/lens2/output-variables

## Data Access

This section shows a detailed description and a step-by-step process of accessing the CESM1 data catalog through Pangeo tuturials on AWS. 

Here we are: 

- Accessing the OCN model dataset with RCP 8.5 and 20C runs. 
- Narrowing down the columns to specific variables we aim to access.

For example: 
``` 
col_ocntemp = col.search(
    frequency=["monthly"],
    component="ocn",
    variable="TEMP",
    experiment=["20C", "RCP85"],  
)
```
- Loading in lat/lon data separately which was not in CESM1 as of March 2024. 

This section works similar to accessing other catalogs, we will be using Zarr. More information on the library below.

https://zarr.readthedocs.io/en/stable/

In [2]:
# Packages needed generally
from matplotlib import pyplot as plt
import xarray as xr
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs

# Packages needed for CESM1 example
import pprint
import intake
import requests
import aiohttp
import s3fs

# Packages needed for CMIP6
import zarr
import gcsfs
import dask

In [3]:
# Open original collection description file: CESM1 LENS
cat_url = "https://ncar-cesm-lens.s3-us-west-2.amazonaws.com/catalogs/aws-cesm1-le.json"
col = intake.open_esm_datastore(cat_url)
col

# Location of file containing grid (lat/lon) information for CESM1 LENS
grid_url="s3://ncar-cesm-lens/ocn/static/grid.zarr"

In [4]:
# Search for the TEMP variable, display the first few entries in the resulting data frame
col.search(variable="TEMP").df

,variable,long_name,component,experiment,frequency,vertical_levels,spatial_domain,units,start_time,end_time,path
0,TEMP,potential temperature,ocn,20C,monthly,60.0,global_ocean,degC,1920-01-16 12:00:00,2005-12-16 12:00:00,s3://ncar-cesm-lens/ocn/monthly/cesmLE-20C-TEM...
1,TEMP,potential temperature,ocn,CTRL,monthly,60.0,global_ocean,degC,0400-01-16 12:00:00,2200-12-16 12:00:00,s3://ncar-cesm-lens/ocn/monthly/cesmLE-CTRL-TE...
2,TEMP,potential temperature,ocn,HIST,monthly,60.0,global_ocean,degC,1850-01-16 12:00:00,1919-12-16 12:00:00,s3://ncar-cesm-lens/ocn/monthly/cesmLE-HIST-TE...
3,TEMP,potential temperature,ocn,RCP85,monthly,60.0,global_ocean,degC,2006-01-16 12:00:00,2100-12-16 12:00:00,s3://ncar-cesm-lens/ocn/monthly/cesmLE-RCP85-T...


In [5]:
# Get more detailed: search for monthly output for the 20th century and RCP8.5 
# ("HIST" is the 1850-1919 period, which is only in the first ensemble member, and "20C" is 1920-2005 which is common across all the other members = 
col_ocntemp = col.search(
    frequency=["monthly"],
    component="ocn",
    variable="SST",
    experiment=["20C", "RCP85"],  
)

col_ocntemp.df

,variable,long_name,component,experiment,frequency,vertical_levels,spatial_domain,units,start_time,end_time,path
0,SST,potential temperature,ocn,20C,monthly,1.0,global_ocean,degC,1920-01-16 12:00:00,2005-12-16 12:00:00,s3://ncar-cesm-lens/ocn/monthly/cesmLE-20C-SST...
1,SST,potential temperature,ocn,RCP85,monthly,1.0,global_ocean,degC,2006-01-16 12:00:00,2100-12-16 12:00:00,s3://ncar-cesm-lens/ocn/monthly/cesmLE-RCP85-S...


In [6]:
# Load catalog entries for subset into a dictionary of xarray datasets
dsets = col_ocntemp.to_dataset_dict(
    zarr_kwargs={"consolidated": True}, storage_options={"anon": True}
)
print(f"\nDataset dictionary keys:\n {dsets.keys()}")


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency'



Dataset dictionary keys:
 dict_keys(['ocn.RCP85.monthly', 'ocn.20C.monthly'])


In [7]:
# Get file containing lat/lon grid information: s3://ncar-cesm-lens/ocn/static/grid.zarr
fs = s3fs.S3FileSystem(anon=True)
grid = xr.open_zarr(fs.get_mapper(grid_url), consolidated=True)

grid

<xarray.Dataset>
Dimensions:               (nlat: 384, nlon: 320, z_t: 60, z_w: 60,
                           lat_aux_grid: 395, moc_comp: 3, moc_z: 61,
                           transport_comp: 5, transport_reg: 2, z_t_150m: 15,
                           z_w_bot: 60, z_w_top: 60)
Coordinates: (12/63)
    ANGLE                 (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    ANGLET                (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    DXT                   (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    DXU                   (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    DYT                   (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    DYU                   (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    ...                    ...
    vonkar                float64 ...
  * z_t                   (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_t_150m              (z_t_150m) float32 500.0 1.5e+03 ... 1.35e+04 1.45e+04
  * z_w                   (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_bot               (z_w_bot) float32 1e+03 2e+03 ... 5.25e+05 5.5e+05
  * z_w_top               (z_w_top) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
Dimensions without coordinates: nlat, nlon, moc_comp, transport_comp,
                                transport_reg
Data variables:
    *empty*

In [8]:
# Define Xarray datasets corresponding to the two experiments
ds_20C = dsets["ocn.20C.monthly"]
ds_RCP85 = dsets["ocn.RCP85.monthly"]

In [9]:
# Merge the datasets
merged_dataset = xr.merge([grid, ds_20C])
ds_20C_merged_mean=merged_dataset.mean(dim="member_id")
ds_20C_merged_mean.SST.sel(time=slice("1950", "2000"))

/Users/diananavarro/opt/anaconda3/envs/cmip6/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


<xarray.DataArray 'SST' (time: 612, z_t: 60, nlat: 384, nlon: 320)>
dask.array<getitem, shape=(612, 60, 384, 320), dtype=float32, chunksize=(300, 60, 384, 320), chunktype=numpy.ndarray>
Coordinates: (12/54)
  * z_t                 (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
    ANGLE               (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    ANGLET              (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    DXT                 (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    DXU                 (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    DYT                 (nlat, nlon) float64 dask.array<chunksize=(192, 160), meta=np.ndarray>
    ...                  ...
    sea_ice_salinity    float64 ...
    sflux_factor        float64 ...
    sound               float64 ...
    stefan_boltzmann    float64 ...
    vonkar              float64 ...
  * time                (time) object 1950-01-16 12:00:00 ... 2000-12-16 12:0...
Dimensions without coordinates: nlat, nlon

In [10]:
#lets plot the SST for the 20th century
fig, ax = plt.subplots(figsize=(12, 6))
ds_20C_merged_mean.SST.sel(time=slice("1950", "2000")).mean(dim="time").plot(ax=ax)
ax.set_title("SST (20th Century)")
plt.show()


: 

: 

## Data Exploration

In this part of the tutorial we will conduct preliminary data exploration to understand the large ensembles we are working with. This data is extremely large, so most of our data exploration is shown through "lazyloading." 

Here we can explore different variables, and the column names.

## Data Wrangling

Although data that is made avaiable through the CESM1 model is clean and ready to use, we only want to extract a certain part of the world. This is the Channel Islands Marine Sanctuary and surrounding areas.  


In this section we will demonstrate how to: 

- Merge the grid data and the CESM1 data from the catalog.
- Use `mask` to subset the data from the Channel Islands, more specifically latitude and longitude. 
- Clarify you have extracted the proper data. 

## Plotting Visuals

Plotting the type of vizualization method, in this section we are mapping the variables. 

This section shows how to properly display the spatial distribution of the variables you are plotting. We will demonstrate some difference plots and maps of data from certain time frames. 